## Imports and Set up


In [61]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


number = 1745113721

directory = "../data/" + str(number)
files = os.listdir(directory)
dfs = []
print(files)

# i = 0
for f in files:
    p = os.path.join(directory, f)
    d = pd.read_csv(p)
    dfs.append(d)
    # i += 1
    # if i > 10:
    #     break

df = pd.concat(dfs, ignore_index=True)


Using device: cpu
['spd1vis1sep1.csv', 'spd1vis1sep3.csv', 'spd1vis1sep5.csv', 'spd1vis1sep7.csv', 'spd1vis1sep9.csv', 'spd1vis5sep1.csv', 'spd1vis5sep3.csv', 'spd1vis5sep5.csv', 'spd1vis5sep7.csv', 'spd1vis5sep9.csv', 'spd1vis9sep1.csv', 'spd1vis9sep3.csv', 'spd1vis9sep5.csv', 'spd1vis9sep7.csv', 'spd1vis9sep9.csv', 'spd1vis13sep1.csv', 'spd1vis13sep3.csv', 'spd1vis13sep5.csv', 'spd1vis13sep7.csv', 'spd1vis13sep9.csv', 'spd1vis17sep1.csv', 'spd1vis17sep3.csv', 'spd1vis17sep5.csv', 'spd1vis17sep7.csv', 'spd1vis17sep9.csv', 'spd3vis1sep1.csv', 'spd3vis1sep3.csv', 'spd3vis1sep5.csv', 'spd3vis1sep7.csv', 'spd3vis1sep9.csv', 'spd3vis5sep1.csv', 'spd3vis5sep3.csv', 'spd3vis5sep5.csv', 'spd3vis5sep7.csv', 'spd3vis5sep9.csv', 'spd3vis9sep1.csv', 'spd3vis9sep3.csv', 'spd3vis9sep5.csv', 'spd3vis9sep7.csv', 'spd3vis9sep9.csv', 'spd3vis13sep1.csv', 'spd3vis13sep3.csv', 'spd3vis13sep5.csv', 'spd3vis13sep7.csv', 'spd3vis13sep9.csv', 'spd3vis17sep1.csv', 'spd3vis17sep3.csv', 'spd3vis17sep5.csv', 'sp

In [62]:
df.head()

,Step,AgentID,NormDir,NeighborInfo,run
0,1,1,[-0.01637289 -0.63029969],"[0.1, 0.05, 0.1, -0.016372894281692307, -0.630...",0
1,1,2,[-0.9602425 0.27916724],"[0.1, 0.05, 0.1, -0.6455055670988146, 0.198085...",0
2,1,3,[-0.96392861 0.26616091],"[0.1, 0.05, 0.1, -0.40212428851208726, 0.16341...",0
3,1,4,[0.31945667 0.94760088],"[0.1, 0.05, 0.1, 0.21060577875371833, 0.850229...",0
4,1,5,[0.95121752 0.30852103],"[0.1, 0.05, 0.1, 0.6497592294313428, 0.2160094...",0


## Prepare Data

In [63]:
# df["NeighborDiffSum"] = df["NeighborDiffSum"].apply(lambda x: eval(x) if isinstance(x, str) else x)
df["NormDir"] = df["NormDir"].apply(lambda s: list(map(float, s.strip('[]').split())))
df["NeighborInfo"] = df["NeighborInfo"].apply(lambda x: eval(x) if isinstance(x, str) else x)



In [64]:

"""
group data according to each time step
"""
# grouped = df.groupby("Step").agg(list)
# neighbor_diff_sum = np.array(grouped["NeighborDiffSum"].to_list())  # Shape: (num_steps, num_agents, 2)
neighbor_info = np.array(df["NeighborInfo"].to_list())  # Shape: (num_steps, num_agents, 2)
normdir = np.array(df["NormDir"].to_list()) #.reshape(len(grouped), len(grouped.iloc[0]["NormDir"]), 1)  # Shape: (num_steps, num_agents, 1)

"""
this makes sure angle difference wraps correctly within [-pi. pi]
"""
# def angle_difference(theta2, theta1):
#     return np.arctan2(np.sin(theta2 - theta1), np.cos(theta2 - theta1))
# delta_angle = np.zeros_like(angles)
# delta_angle[1:] = angle_difference(angles[1:], angles[:-1]) # Shape: (num_steps, num_agents, 1)

# grouped["deltaAngle"] = delta_angle.reshape(len(grouped), -1).tolist()

# X_train, X_test, y_train, y_test = train_test_split(neighbor_diff_sum, delta_angle, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(neighbor_info, normdir, test_size=0.2, random_state=42)

# to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

## For Debugging

In [154]:
normdir.mean(axis=0)

array([-0.06299599, -0.00369139])

In [151]:
normdir.shape

(1500000, 2)

In [130]:
df['NeighborInfo'].head()

0    [0.1, 0.05, 0.1, -0.016372894281692307, -0.630...
1    [0.1, 0.05, 0.1, -0.6455055670988146, 0.198085...
2    [0.1, 0.05, 0.1, -0.40212428851208726, 0.16341...
3    [0.1, 0.05, 0.1, 0.21060577875371833, 0.850229...
4    [0.1, 0.05, 0.1, 0.6497592294313428, 0.2160094...
Name: NeighborInfo, dtype: object

In [129]:
df['NormDir'].head()

0    [-0.01637289, -0.63029969]
1      [-0.9602425, 0.27916724]
2     [-0.96392861, 0.26616091]
3      [0.31945667, 0.94760088]
4      [0.95121752, 0.30852103]
Name: NormDir, dtype: object

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Compute incorrect delta angles (naive subtraction)
# naive_delta_angle = np.zeros_like(angles)
# naive_delta_angle[1:] = angles[1:] - angles[:-1]

# # Compute corrected delta angles (using atan2)
# corrected_delta_angle = np.zeros_like(angles)
# corrected_delta_angle[1:] = np.arctan2(np.sin(angles[1:] - angles[:-1]), np.cos(angles[1:] - angles[:-1]))

# # Plot angles over time
# plt.figure(figsize=(12, 6))

# plt.subplot(3, 1, 1)
# plt.plot(angles[:, 3, 0], label="Angle (Agent 1)")
# plt.xlabel("Time Step")
# plt.ylabel("Angle (radians)")
# plt.title("Angle Over Time")
# plt.legend()

# plt.subplot(3, 1, 2)
# plt.plot(naive_delta_angle[:, 1, 0], label="Naive Delta Angle", color="red")
# plt.xlabel("Time Step")
# plt.ylabel("Δ Angle")
# plt.title("Naive Delta Angle (No Wrap Handling)")
# plt.legend()

# plt.subplot(3, 1, 3)
# plt.plot(corrected_delta_angle[:, 1, 0], label="Corrected Delta Angle", color="green")
# plt.xlabel("Time Step")
# plt.ylabel("Δ Angle")
# plt.title("Corrected Delta Angle (With atan2 Handling)")
# plt.legend()

# plt.tight_layout()
# plt.show()


In [6]:
# normdir_df = pd.DataFrame(normdir.reshape(len(grouped), -1))  # col: agent number; row: timestep
# delta_angle_df = pd.DataFrame(delta_angle.reshape(len(grouped), -1))  # Each column is an agent's deltaAngle

NameError: name 'grouped' is not defined

In [ ]:
# normdir_df = pd.DataFrame(

In [ ]:
# normdir_df.head(30)

In [ ]:
# delta_angle_df.head(10)

In [ ]:
np.arctan2(-0.09105, 0.9958)

## Define Model

In [116]:
# define nn
hidden_size_1 = 24
hidden_size_2 = 8
hidden_size_3 = 4

model = nn.Sequential(
    nn.Linear(17, hidden_size_1),
    nn.ReLU(),
    nn.Linear(hidden_size_1, hidden_size_1),
    nn.ReLU(),
    nn.Linear(hidden_size_1, hidden_size_2),
    nn.ReLU(),
    nn.Linear(hidden_size_2, 2)
)

# define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Define Model

In [147]:
optimizer = optim.Adam(model.parameters(), lr=0.002)

In [148]:
num_epochs = 500
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor.view(-1, 17))
    loss = criterion(outputs, y_train_tensor.view(-1, 2))
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

Epoch [10/500], Loss: 0.073461
Epoch [20/500], Loss: 0.065997
Epoch [30/500], Loss: 0.064614
Epoch [40/500], Loss: 0.063342
Epoch [50/500], Loss: 0.062947
Epoch [60/500], Loss: 0.062825
Epoch [70/500], Loss: 0.062773
Epoch [80/500], Loss: 0.062746
Epoch [90/500], Loss: 0.062731
Epoch [100/500], Loss: 0.062722
Epoch [110/500], Loss: 0.062714
Epoch [120/500], Loss: 0.062707
Epoch [130/500], Loss: 0.062700
Epoch [140/500], Loss: 0.062694
Epoch [150/500], Loss: 0.062687
Epoch [160/500], Loss: 0.062681
Epoch [170/500], Loss: 0.062674
Epoch [180/500], Loss: 0.062668
Epoch [190/500], Loss: 0.062661
Epoch [200/500], Loss: 0.062654
Epoch [210/500], Loss: 0.062648
Epoch [220/500], Loss: 0.062640
Epoch [230/500], Loss: 0.062633
Epoch [240/500], Loss: 0.062626
Epoch [250/500], Loss: 0.062619
Epoch [260/500], Loss: 0.062611
Epoch [270/500], Loss: 0.062603
Epoch [280/500], Loss: 0.062596
Epoch [290/500], Loss: 0.062588
Epoch [300/500], Loss: 0.062580
Epoch [310/500], Loss: 0.062573
Epoch [320/500], 

## Evaluate the model

In [149]:
with torch.no_grad():
    y_pred = model(X_test_tensor.view(-1, 17))
    test_loss = criterion(y_pred, y_test_tensor.view(-1, 2))
    print(f'Test Loss: {test_loss.item():.6f}')
    print(y_pred[0], y_test_tensor.view(-1, 2)[0], X_test_tensor.view(-1, 17)[0][3:5])
    print(y_pred[1], y_test_tensor.view(-1, 2)[1], X_test_tensor.view(-1, 17)[1][3:5])
    print(y_pred[2], y_test_tensor.view(-1, 2)[2], X_test_tensor.view(-1, 17)[2][3:5])
    print(y_pred[3], y_test_tensor.view(-1, 2)[3], X_test_tensor.view(-1, 17)[3][3:5])
    print(y_pred[4], y_test_tensor.view(-1, 2)[4], X_test_tensor.view(-1, 17)[4][3:5])

Test Loss: 0.062851
tensor([-0.6171, -0.7505]) tensor([-0.7677, -0.6408]) tensor([-0.8116, -0.7595])
tensor([-0.0079, -0.9626]) tensor([-0.3465, -0.9380]) tensor([ 0.1153, -0.9933])
tensor([0.1740, 0.8812]) tensor([0.3264, 0.9452]) tensor([0.2517, 0.9678])
tensor([0.2458, 0.6504]) tensor([-0.1676,  0.9859]) tensor([0.8600, 0.5102])
tensor([-0.6533, -0.8197]) tensor([-0.6060, -0.7955]) tensor([-0.6060, -0.7955])


## Save the model

In [150]:
torch.save(model, '../models/20250419_v5')